In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from keras import models
from keras import layers
import tensorflow as tf
from Informer import Informer
import warnings
warnings.filterwarnings('ignore')

2025-09-29 15:38:31.947954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759147711.964766   29313 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759147711.970356   29313 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759147711.983157   29313 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759147711.983176   29313 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759147711.983178   29313 computation_placer.cc:177] computation placer alr

In [2]:
def create_sequences(X, y, date, window_size):
    seq_X = []
    seq_y = []
    seq_date = []
    for i in range(len(X) - window_size):
        seq_X.append(X[i:i+window_size])
        seq_date.append(date[i:i+window_size])
        seq_y.append(y[i+window_size])

    return np.array(seq_X), np.array(seq_y), np.array(seq_date)

In [3]:
# read dataset
df= pd.read_csv('Data/Tetuan City power consumption.csv')

In [4]:
# create date dataset
df_date= pd.DataFrame()
df['DateTime']= pd.to_datetime(df['DateTime'])
df_date['minute']= df['DateTime'].dt.minute
df_date['hour']= df['DateTime'].dt.hour
df_date['weekday']= df['DateTime'].dt.weekday
df_date['month']= df['DateTime'].dt.month

In [5]:
# create and prepare data & target 
df_data= df.drop(['DateTime', 'Zone 1 Power Consumption'], axis=1)
df_target= df[['Zone 1 Power Consumption']]

# power scaler
Power_scaler= PowerTransformer(method='box-cox')
df_data['normed_general diffuse flows'] = Power_scaler.fit_transform(df_data[['general diffuse flows']])
df_data['normed_diffuse flows'] = Power_scaler.fit_transform(df_data[['diffuse flows']])
df_data['normed_humidity'] = np.clip(df_data['Humidity'], a_min=40, a_max=90)
df_data = df_data.drop(['Humidity', 'general diffuse flows', 'diffuse flows'], axis=1)

# MinMax scaler
MinMax_scaler= MinMaxScaler(feature_range=(0, 1))
df_data= MinMax_scaler.fit_transform(df_data)
df_target= MinMax_scaler.fit_transform(df_target)

# sequencing data
data, target, date= create_sequences(df_data, df_target, df_date.values, window_size=144)

In [6]:
# prepare date
date_df = pd.DataFrame(date.reshape(-1, date.shape[-1]),
                       columns=['minute', 'hour', 'weekday', 'month'])


date_used = date_df[['hour', 'weekday', 'month']].values
date_used = date_used.reshape(date.shape[0], date.shape[1], 3)

dataset = tf.data.Dataset.from_tensor_slices(((data, date_used), target))

# train/val/test split
train_size = int(0.7 * len(data))
val_size = int(0.15 * len(data))

train_ds = dataset.take(train_size).batch(32).shuffle(100)
val_ds = dataset.skip(train_size).take(val_size).batch(32)
test_ds = dataset.skip(train_size + val_size).batch(32)

I0000 00:00:1759147714.932021   29313 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2541 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
2025-09-29 15:38:34.933407: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 421521408 exceeds 10% of free system memory.
2025-09-29 15:38:35.323936: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90326016 exceeds 10% of free system memory.
2025-09-29 15:38:35.367023: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 421521408 exceeds 10% of free system memory.


In [7]:
# تعریف مدل
model = Informer(d_model=64, num_heads=4)  # مقادیر رو بسته به منابع تغییر بده
model.compile(optimizer='adam', loss='mse')

# آموزش
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10)

# ارزیابی
model.evaluate(test_ds)

Epoch 1/10


2025-09-29 15:38:35.684725: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 421521408 exceeds 10% of free system memory.
2025-09-29 15:38:35.848810: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90326016 exceeds 10% of free system memory.


TypeError: missing a required argument: 'context_time'